In [84]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from numpy import kron
from random import random
from math import pi
from scipy.optimize import minimize    

All the necessary matrices and gates for the VQE-circuit are recreated here:

In [85]:
np.set_printoptions(precision=4,suppress=True)

#Pauli matrices
I  = np.array([[ 1, 0],
               [ 0, 1]])
Sigma_x = np.array([[ 0, 1],
               [ 1, 0]])
Sigma_y = np.array([[ 0,-1j],
               [1j, 0]])
Sigma_z = np.array([[ 1, 0],
               [ 0,-1]])

#Hadamard matrix
Hdm = (1/np.sqrt(2))*np.array([[ 1, 1],[ 1, -1]])

#Phase matrix
S = np.array([[ 1, 0],[ 0, 1j]])

#Single qubit basis states |0> and |1>
q0 = np.array([[1],[0]])
q1 = np.array([[0],[1]])

#Rotation matrices, as theta functions
Rx = lambda theta : np.array([[    np.cos(theta/2),-1j*np.sin(theta/2)],[-1j*np.sin(theta/2),    np.cos(theta/2)]])
Ry = lambda theta : np.array([[    np.cos(theta/2),   -np.sin(theta/2)],[    np.sin(theta/2),    np.cos(theta/2)]])
Rz = lambda theta : np.array([[np.exp(-1j*theta/2),                0.0],[                0.0, np.exp(1j*theta/2)]])

#Projection matrices |0><0| and |1><1|
P0  = np.dot(q0,q0.conj().T)
P1  = np.dot(q1,q1.conj().T)

#CNOTij, where i is control qubit and j is target qubit
CNOT01 = np.kron(I,P0) + np.kron(Sigma_x,P1) # control -> q0, target -> q1

#Our initial state 

psi_0 = np.zeros((4,1))
psi_0[0] = 1

# Our Hamiltonian 
H = [[1, 0, 0, 0],
    [0, 0, -1, 0],
    [0, -1, 0, 0],
    [0, 0, 0, 1]]


The next part is not a generic type of code for the VQE problem but specific solution for the given Hamiltonian, in order to derive the coefficients of the decomposed form

In [86]:
def HS(M1, M2):
    #Hilbert-Schmidt-Product of two matrices M1, M2
    return (np.dot(M1.conj().T, M2)).trace()


def c2s(c):
    #Return a string representation of a complex number c
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)
    
def decompose(H):
    #Decompose Hermitian 4x4 matrix H into Pauli matrices
    
    Sigma = [I, Sigma_x, Sigma_y, Sigma_z]
    labels = ['I', 'σ_x', 'σ_y', 'σ_z']
    for i in range(4):
        for j in range(4):
            label = labels[i] + ' /otimes ' + labels[j]
            a_ij = 0.25 * HS(np.kron(Sigma[i], Sigma[j]), H)
            if a_ij != 0.0:
                print ("%s\t*\t( %s )" % (c2s(a_ij), label))
                
              
            
decompose(H)            

0.5	*	( I /otimes I )
-0.5	*	( σ_x /otimes σ_x )
-0.5	*	( σ_y /otimes σ_y )
0.5	*	( σ_z /otimes σ_z )


In [87]:
#The decomposed form (not a generalized case) and derive the coefficients

a = +0.5
b = -0.5
c= -0.5
d = +0.5

H_decomposed = (a * np.kron( I, I) + b * np.kron(Sigma_z,Sigma_z) + c * np.kron(Sigma_y,Sigma_y) + d * np.kron(Sigma_x,Sigma_x))  

exact_energy = np.linalg.eigvalsh(H)[0]

Here is the main calculation of the expectation value via the VQE. This was an approach based on unitary transformation of basic Pauli measurements. Any unitary transformation can either act on the operator or on the state, the final result is derived by applying the chosen transformation and then measuring Z, thus rendering possible the acquisition of any Pauli measurement you want.

In [88]:
#Choosing our ansatz (changes in the ansatz yields different results)

ansatz = lambda theta: (np.dot(np.dot(np.kron(Rx(np.pi/2), I),np.dot(CNOT10, np.dot(np.kron(I,Rz(theta)),
                    CNOT10))), np.kron(Hdm,I)))

def VQE(theta,ansatz,psi0):
    #This will depend on the Hamiltonian + coefficients
    circuit = ansatz(theta[0])
    psi = np.dot(circuit,psi_0)
    
    #For 2 qubits, assume we can only take Pauli Sigma_z measurements np.kron(X, I)
    measureZ = lambda U: np.dot(np.conj(U).T,np.dot(np.kron(Sz,I),U))
    
    expectation_value = 0.0
    
    #These are the unitary transformations of the Pauli measurements 
    
    #For Identity
    expectation_value += a 
    
    #For Pauli - Z
    U = CNOT01
    expectation_value += b * np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    #For Pauli - X
    U = np.dot(CNOT01,np.kron(Hdm,Hdm))
    expectation_value += c * np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    #For Pauli - Y
    U = np.dot(CNOT01,np.kron(np.dot(Hdm,S.conj().T),np.dot(Hdm,S.conj().T)))
    expectation_value += d * np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    return np.real(expectation_value)

theta  = [0.0]
result = minimize(projective_expected,theta,args=(ansatz,psi_0))
theta  = result.x
val    = result.fun

print("VQE: ")
print(theta)
print(val + exact_energy)

VQE: 
[1.5708]
-0.9999999999997558
